In [132]:
import numpy as np
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
import cv2

# %matplotlib inline
%matplotlib qt

%config InlineBackend.figure_format = 'retina'

In [133]:
def load_image():    
    path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-1\A1data\data\compEx4.jpg'
    return cv2.imread(path)

In [134]:
def mat_to_np():    
    path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-1\A1data\data\compEx4.mat'
    mat_data = loadmat(path)
    corners = np.array(mat_data['corners'])
    K = np.array(mat_data['K'])
    v = np.array(mat_data['v'])
    return corners, K ,v

In [135]:
def to_cart(x):
    return x/x[-1]

In [136]:
def normalize(v):
    return v/(v @ v)**0.5

In [137]:
def compute_camera_center(P):
    M = P[:,:3]
    P4 = P[:,-1]
    C = -1*(np.linalg.inv(M) @ P4)
    return C

In [138]:
def compute_principal_axis(P):
    M = P[:,:3]
    m3 = P[-1,:3]
    v = np.linalg.det(M) * m3
    return normalize(v)

In [139]:
def camera_center_and_axis(P):
    C = compute_camera_center(P)
    p_axis = compute_principal_axis(P) 
    return C, p_axis

In [140]:
def calibrate(p, K):
    return np.linalg.inv(K) @ p

In [141]:
def compute_3D_point(pi, x2D):
    s = -np.transpose(pi[:-1]) @ x2D
    x3D = x2D / s
    return x3D

In [142]:
def plot_original_2D(corners, img, plt_img=False):
    x = corners[0,:]
    y = corners[1,:]

    fig = plt.figure(figsize=(10,8))
    plt.plot(x, y, 'o', color='red', label='Corners')
    plt.axis('equal')
    plt.xlabel('$x$')
    plt.ylabel('$y$')

    if plt_img is True:
        plt.legend(loc="lower right")
        plt.imshow(img)
        fig.savefig(r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-1\report-images\E4_image_and_corners_original.png', dpi=300)
    else:
        plt.gca().invert_yaxis()
        plt.legend(loc="lower right")
        fig.savefig(r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-1\report-images\E4_normalised_corners_original.png', dpi=300)

    plt.show()

In [143]:
def plot_3D(P1, P2, s1, s2, U1, U2):
    C1, p_axis1 = camera_center_and_axis(P1)
    C2, p_axis2 = camera_center_and_axis(P2)

    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection='3d')

    ax.plot(U1[0], U1[1], U1[2], 'o', color='magenta', label='Corners 3D')
    ax.plot(U2[0], U2[1], U2[2], 'o', color='green', label='Corners 2D')

    ax.plot(C1[0], C1[1], C1[2], 'o', color='red', label='Camera center 1')
    ax.plot(C2[0], C2[1], C2[2], 'o', color='lime', label='Camera center 2')

    x_p_axis1 = C1[0] + s1*p_axis1[0]
    y_p_axis1 = C1[1] + s1*p_axis1[1]
    z_p_axis1 = C1[2] + s1*p_axis1[2]

    x_p_axis2 = C2[0] + s2*p_axis2[0]
    y_p_axis2 = C2[1] + s2*p_axis2[1]
    z_p_axis2 = C2[2] + s2*p_axis2[2]

    ax.plot(x_p_axis1, y_p_axis1, z_p_axis1, 'o', color='blue', label='$s_1\\cdot$Principal axis 1')
    ax.plot(x_p_axis2, y_p_axis2, z_p_axis2, 'o', color='cyan', label='$s_2\\cdot$Principal axis 2')

    ax.plot([x_p_axis1,C1[0]], [y_p_axis1,C1[1]], [z_p_axis1,C1[2]], '-', color='black')
    ax.plot([x_p_axis2,C2[0]], [y_p_axis2,C2[1]], [z_p_axis2,C2[2]], '-', color='black')

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.axis('equal')

    plt.legend(loc="lower right")
    fig.savefig(r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-1\report-images\E4_3D-3.png', dpi=300)

    plt.show()
    

In [144]:
def get_camera_and_scale_1():
    P1 = np.concatenate((np.identity(3), np.zeros([3,1])), 1)
    s1 = 3
    return P1, s1

In [145]:
def get_camera_and_scale_2():
    cos = np.cos(np.pi/6)
    sin = np.sin(np.pi/6)

    R = np.array([[cos,0,-sin],[0,1,0],[sin,0,cos]])
    C = np.array([-2,0,0])
    p4 = np.expand_dims((-C @ np.transpose(R)), 1)

    P2 = np.concatenate((R, p4), 1)
    s2 = 3.5
    return P2, s2

In [146]:
def compute_homography(P2, pi, K):
    R = P2[:,:3]
    t = P2[:,-1]
    H = R - np.outer(t, np.transpose(pi[:-1]))
    H_tot = K @ H @ np.linalg.inv(K)
    return H, H_tot

In [147]:
def transform_img(img, H_tot, corners):
    width = np.size(img, 1)
    height = np.size(img, 0)
    transf_img = cv2.warpPerspective(img, H_tot, (width, height))
    corners_transf = H_tot @ corners
    return transf_img, to_cart(corners_transf)

In [150]:
def plot_transformed_2D(img, corners_transf, y_H, y_P, plt_norm=False):

    if plt_norm is True:
        fig = plt.figure(figsize=(10,8))
        plt.plot(y_H[0,:], y_H[1,:], 'D', color='blue', label='$y=Hx$')
        plt.plot(y_P[0,:], y_P[1,:], '.', color='magenta', label='$y=P_2X$')
        plt.axis('equal')
        plt.gca().invert_yaxis()
        plt.legend(loc="lower right")
        plt.xlabel('$x$')
        plt.ylabel('$y$')
        fig.savefig(r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-1\report-images\E4_normalised_corners_transformed.png', dpi=300)
    
    else:
        fig = plt.figure(figsize=(10,8))
        plt.plot(corners_transf[0,:], corners_transf[1,:], 'o', color='red', label='$y=H_{tot}x$')
        plt.axis('equal')
        plt.legend(loc="lower right")
        plt.xlabel('$x$')
        plt.ylabel('$y$')
        plt.imshow(img)
        fig.savefig(r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-1\report-images\E4_image_and_corners_transformed.png', dpi=300)
        
    plt.show()

In [149]:
img = load_image()
corners, K, v = mat_to_np()
corners_cal = calibrate(corners, K)

plot_original_2D(corners, img, plt_img=True)
plot_original_2D(corners_cal, img)

corners_R3 = np.transpose(np.array([compute_3D_point(to_cart(v), corners_cal[:,i]) for i in range(np.size(corners_cal, 1))]))

P1, s1 = get_camera_and_scale_1()
P2, s2 = get_camera_and_scale_2()

plot_3D(P1, P2, s1, s2, corners_R3, corners_cal)

corners_P3 = np.concatenate((corners_R3, np.ones([1,4])), 0)
H, H_tot = compute_homography(P2, to_cart(v), K)

y_H = to_cart(H @ corners_cal)
y_P = to_cart(P2 @ corners_P3)

transf_img, corners_transf = transform_img(img, H_tot, corners)

plot_transformed_2D(transf_img, corners_transf, y_H, y_P, plt_norm=True)
plot_transformed_2D(transf_img, corners_transf, y_H, y_P)